### Data source: https://www.bracs.icar.cnr.it/

### Import packages

In [ ]:
import numpy as np 
import pandas as pd 
import os
import cv2
import random
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
from collections import Counter
from sklearn.preprocessing import LabelBinarizer, LabelEncoder

import torch
import torchvision
from PIL import Image
from torch.utils.data import Dataset
import torch.nn as nn
from torchvision.io import read_image
from torchvision.models import resnet50, ResNet50_Weights
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import transforms
from torch.utils.data.sampler import Sampler
import json

from sklearn.metrics import f1_score, precision_recall_curve
from sklearn.metrics import classification_report

import timm

from omegaconf import OmegaConf
# Load config
preproc_conf = OmegaConf.load("../conf/preproc.yaml")
preproc_conf = preproc_conf['classic_mil_on_embeddings_bag']['bracs_224_224_patches']

### Locate annotations

In [ ]:
parent_folder = preproc_conf.img_dir_lvl4
parent_folder

### Load data

In [ ]:
%%time

## HERE CHOOSE NO NORM OR MACENKO NORM:
#data = np.load( parent_folder+'bracs_level4_regions_224_data.npy')
data = np.load( parent_folder+'bracs_level4_regions_224_data_macenkonorm_bracs.npy')

## load this to be able to gen confusion matrices:
data_macenko = np.load( parent_folder+'bracs_level4_regions_224_data_macenkonorm_bracs.npy')

label = np.load( parent_folder+'bracs_level4_regions_224_label.npy')
data.shape, label.shape

In [ ]:
label

### Preprocess label

In [ ]:
#Binary encode
lb = LabelEncoder()
#lb = LabelBinarizer()
lb.fit(label)
label_oh = lb.transform(label)

In [ ]:
label_oh.shape

In [ ]:
lb.classes_

In [ ]:
Counter(label_oh)

### Create balanced data loader -> balanced folds with subset

In [ ]:
def create_balanced_biopsy_subset(labels, minority_class_ratio=0.2, rnd_seed=38):
    # set random seed as given
    np.random.seed(rnd_seed)
    
    # collect selected biopsies that will be in the balanced subset
    test_local_idx = []
    
    # get current class occurences for biopsy
    class_occurence = np.array(list(dict( Counter(labels) ).values()))[ np.argsort(list(dict( Counter(labels) ).keys()))]
    #print(class_occurence)
    
    # calc class weights
    class_weights = ( class_occurence / class_occurence.sum() ).astype(np.float32)
    class_weights_dict = dict( zip( np.arange(class_weights.shape[0]), class_weights ))
    #print(class_weights_dict)
    
    # how many of biopsies to include in the balanced subset
    nr_class_test = int(labels.shape[0]*np.min(class_weights)*minority_class_ratio)

    # collect biopsy indices for the balanced subset
    for s in np.unique(labels): #loop over labelss
        s_idx = np.arange(labels.shape[0])[labels == s]
        rnd_idx = np.random.permutation(s_idx.shape[0])
        test_local_idx.append(s_idx[rnd_idx[:nr_class_test]])

    # aggregate all the balanced subset's indices
    test_idx = np.concatenate(test_local_idx)
    
    # other indices not in balanced set will be the rest
    train_idx = np.arange(labels.shape[0])[~np.in1d(np.arange(labels.shape[0]), test_idx)]
    
    return train_idx, test_idx#, label_remaining[]

In [ ]:
remaining_idx, val_idx = create_balanced_biopsy_subset(label)

In [ ]:
X_val = data[val_idx]
y_val = label[val_idx]
y_val = lb.transform(y_val)

X_remaining = data[remaining_idx]
y_remaining = label[remaining_idx]
y_remaining = lb.transform(y_remaining)

X_val.shape, y_val.shape, X_remaining.shape, y_remaining.shape

In [ ]:
X_val_macenko = data_macenko[val_idx]
X_remaining_macenko = data_macenko[remaining_idx]

X_val = data[val_idx]
X_remaining = data[remaining_idx]

In [ ]:
def give_back_balanced_training_fold( X_current, y_current,
                                      minority_class_ratio=0.25, rnd_seed=12 ):
    
    _, test_idx, = create_balanced_biopsy_subset(y_current,
                                                 minority_class_ratio,
                                                 rnd_seed)
    X_train_balanced = X_current[test_idx]
    y_train_balanced = y_current[test_idx]
    #y_train_balanced_oh = lb.transform(y_train_balanced)
    #print( X_train_balanced.shape, y_train_balanced_oh.shape )
    
    return X_train_balanced, y_train_balanced

In [ ]:
X_train, y_train = give_back_balanced_training_fold(X_remaining, y_remaining, rnd_seed=15)
X_train.shape, y_train.shape

## Dataloader

In [ ]:
class CollectionsDataset(Dataset):
    def __init__(self,
                 data,
                 labels,
                 num_classes, 
                 transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform
        self.num_classes = num_classes

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = self.data[idx]
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)

        return {'image': image,
                'label': label
                }

In [ ]:
#https://github.com/pytorch/pytorch/issues/7359
class BalancedSampler(Sampler):
    def __init__(self,
                 batch_size,
                 data,
                 labels,
                 num_classes, 
                 transform=None,
                 rand_seed=12):
        self.batch_size = batch_size
        self.data = data
        self.labels = labels
        self.transform = transform
        self.num_classes = num_classes
        self.rand_seed = rand_seed
        

        
    def give_back_balanced_training_fold(self, X_current, y_current, minority_class_ratio, rand_seed):
        _, test_idx = create_balanced_biopsy_subset(y_current, minority_class_ratio, rand_seed)
        
        return test_idx
        

    def __len__(self):
        test_idx = self.give_back_balanced_training_fold(self.data, self.labels, minority_class_ratio=0.25, rand_seed=self.rand_seed)
        
        return len(test_idx) #3220

    def __iter__(self):
        test_idx = self.give_back_balanced_training_fold(self.data, self.labels, minority_class_ratio=0.25, rand_seed=self.rand_seed)
        
        random.shuffle(test_idx)
        num_batches = len(test_idx) // self.batch_size - 1 #99
        
        n=0
        while num_batches > 0:
            
            sampled = test_idx[n*32:(n+1)*32]
                
            yield sampled #(32, 3, 224, 224)
            num_batches -=1
            n += 1

In [ ]:
class BatchBalancedSampler(Sampler):

    def __init__(self, sampler, batch_size):
        self.sampler = sampler
        self.batch_size = batch_size

    def __iter__(self):
        for _, idx in enumerate(iter(self.sampler)): #99
            batch = idx
            yield batch

    def __len__(self):
        return len(self.sampler) // self.batch_size - 1 #99

## Model

#### frozen backbone

#### very small LR for backbone

In [ ]:
# Load UNI model
model = timm.create_model(
    "vit_large_patch16_224", img_size=224, patch_size=16, init_values=1e-5, num_classes=0
)
model.load_state_dict(torch.load(preproc_conf.uni_embedder_weights, map_location="cpu"), strict=True)
# Modify head to match this task
#model.head = nn.Linear(in_features=1024, out_features=7)

model.head = nn.Sequential(
    nn.Linear(in_features=1024, out_features=128),
    nn.ReLU(),    
    nn.Linear(in_features=128, out_features=7))

# Define two sets of parameters: one for the backbone and one for the head
backbone_params = [param for name, param in model.named_parameters() if 'head' not in name]
head_params = model.head.parameters()

## Traning loop

In [ ]:
weights_folder = 'weights_train_uni_smallLRbackbone_complexhead_level4_macenko_bracs_100epochs/'

os.makedirs(weights_folder, exist_ok=True)

def train_model(model,
                device,
                transform,
                optimizer, 
                scheduler, 
                num_epochs):
    
    history = {}
    history_train_loss = []
    history_train_acc = []
    history_val_loss = []
    history_val_acc = []
    

    criterion = nn.CrossEntropyLoss()
    
    
    train_dataset = CollectionsDataset(data=X_remaining,
                                       labels=y_remaining,
                                       num_classes=NUM_CLASSES,
                                       transform=transform)
    
    # VAL dataset
    val_dataset = CollectionsDataset(data=X_val,
                                     labels=y_val,
                                     num_classes=NUM_CLASSES,
                                     transform=transform)

    # create the pytorch data loader
    val_dataset_loader = torch.utils.data.DataLoader(val_dataset,
                                                     batch_size=BATCH_SIZE,
                                                     shuffle=True,
                                                     num_workers=4)
    
    
    
    # training loop wiht balanced folds
    for epoch in range(0, num_epochs):
        
        sampler = BalancedSampler(
                     batch_size=32,
                     data=X_remaining,
                     labels=y_remaining,
                     num_classes=NUM_CLASSES, 
                     transform=None,
                     rand_seed=int(epoch*1.5+3*epoch))

        batch_sampler = BatchBalancedSampler(sampler, batch_size=BATCH_SIZE)


        # create the pytorch data loader
        train_dataset_loader = torch.utils.data.DataLoader(train_dataset,
                                                           num_workers=4,
                                                           batch_sampler=batch_sampler)
        
        
        
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        

        #scheduler.step()
        model.train()

        running_loss = 0.0
        correct = 0
        total = 0
        
        # Iterate over data.
        
        
        for bi, d in enumerate(train_dataset_loader):
            
            inputs = d["image"]
            labels = d["label"]
            inputs = inputs.to(device, dtype=torch.float, non_blocking=True)
            labels = labels.to(device, dtype=torch.long,  non_blocking=True)

            optimizer.zero_grad()

            with torch.set_grad_enabled(True):
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            
            #acc metrics
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
        checkpoint = { 
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            #'scheduler': scheduler
        }
            
        epoch_loss = running_loss / (len(sampler) -52)
        writer.add_scalar("Train CE Loss", epoch_loss)
        print('Train CE Loss: {:.4f}'.format(epoch_loss), f'----- Train Accuracy: {100 * correct // total} %')
        
        
        history_train_loss.append(epoch_loss)
        history_train_acc.append(100 * correct // total)
        
        
        # VALIDATION
        correct_val = 0
        total_val = 0
        running_loss_val = 0
        # since we're not training, we don't need to calculate the gradients for our outputs
        with torch.no_grad():
            for bi, d in enumerate(val_dataset_loader):
                inputs = d["image"]
                labels = d["label"]
                inputs = inputs.to(device, dtype=torch.float)
                labels = labels.to(device, dtype=torch.long)
                # calculate outputs by running images through the network
                outputs = model(inputs)
                # the class with the highest energy is what we choose as prediction
                _, predicted = torch.max(outputs.data, 1)
                total_val += labels.size(0)
                correct_val += (predicted == labels).sum().item()
                
                loss_val = criterion(outputs, labels)
                
                running_loss_val += loss_val.item() * inputs.size(0)
                
        epoch_loss_val = running_loss_val / X_val.shape[0]
        print('Val CE Loss: {:.4f}'.format(epoch_loss_val), f'----- Val Accuracy: {100 * correct_val // total_val} %')
        print('-' * 10)
        
        history_val_loss.append(epoch_loss_val)
        history_val_acc.append(100 * correct_val // total_val)
        
        if (100 * correct_val // total_val) > 59:
            print(weights_folder+f'checkpoint_epoch_{epoch}'+\
                   '_{:.4f}'.format(epoch_loss_val)+f'_{100 * correct_val // total_val} %.pth')
            torch.save(checkpoint, weights_folder+f'checkpoint_epoch_{epoch}'+\
                   '_{:.4f}'.format(epoch_loss_val)+f'CE_{100 * correct_val // total_val}_acc.pth')

    history['train_loss'] = history_train_loss
    history['train_acc'] = history_train_acc
    history['val_loss'] = history_val_loss
    history['val_acc'] = history_val_acc
    
    writer.flush()
        
        
    return model, history

## Transforms, augmentation

In [ ]:
# check augmentations

In [ ]:
dummy_transform = transforms.Compose([transforms.ToTensor(), transforms.RandomResizedCrop(size=224, scale=(0.55,1))])

dummy_transform2 = transforms.Compose([transforms.ToTensor(), transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1)])

dummy_transform3 = transforms.Compose([transforms.ToTensor(), transforms.RandomAdjustSharpness(sharpness_factor=3, p=0.5)])

dummy_transform4 = transforms.Compose([transforms.ToTensor(), transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 3))])


dummy_dataset = CollectionsDataset(data=X_remaining,
                                       labels=y_remaining,
                                       num_classes=7,
                                       transform=dummy_transform4)

plt.imshow(np.swapaxes(dummy_dataset[0]['image'],0,2))

In [ ]:
dummy_dataset = CollectionsDataset(data=X_remaining,
                                       labels=y_remaining,
                                       num_classes=7,
                                       transform=None)
plt.imshow(dummy_dataset[0]['image'])

In [ ]:
# https://stackoverflow.com/questions/51677788/data-augmentation-in-pytorch

# define some re-usable stuff
IMAGE_SIZE = 224
NUM_CLASSES = 7
BATCH_SIZE = 32
device = torch.device("cuda:0")


train_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)), # IMAGENET !
     #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
     transforms.RandomResizedCrop(size=224, scale=(0.55,1)),
     #transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
     transforms.RandomAdjustSharpness(sharpness_factor=3, p=0.5),
     transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 3))
    ])

# push model to device
model = model.to(device)

### Train UNI with balanced dataset

## Optimizer and scheduler

In [ ]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [ ]:
def seed_torch(seed=7):
    import random
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
    if device.type == 'cuda:0':
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed) # if you are using multi-GPU.
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

In [ ]:
optimizer_ft = optim.Adam(model.parameters(), lr=1e-4, amsgrad=True) # was 1e-4
#optimizer_ft = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

"""
# Set a very small learning rate for the backbone and a higher one for the head
optimizer_ft = optim.Adam([
    {'params': backbone_params, 'lr': 1e-6, 'amsgrad': True},  
    {'params': head_params, 'lr': 1e-4, 'amsgrad': True}      
])
"""

lr_sch = None # lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.8) # was 10 and 0.8
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') #device config

seed_torch() # FIX SEED 
model_ft, history = train_model(model,
                       device,
                       train_transform,
                       optimizer_ft,
                       lr_sch,
                       num_epochs=300)

In [ ]:
plt.figure(figsize=(12,8))
plt.plot( np.arange(len(history['train_loss'])), history['train_loss'], label='train' )
plt.plot( np.arange(len(history['val_loss'])), history['val_loss'], label='val' )
plt.legend(fontsize=18)
plt.xlabel('epochs', fontsize=22)
plt.ylabel('Cross entropy loss', fontsize=22)
plt.tick_params(labelsize=18)
#plt.yscale('log')
plt.title('mcr=0.5, bs=32, AdamW, lr=1e-4, StepLR(step_size=5, gamma=0.8)')
#plt.savefig('test3_aug3')

### Save plot data as well !!

### Nonorm

#### frozen backbone and linear head

#### frozen backbone and complex head

#### finetune backbone as well and linear head

### Macenko

#### frozen backbone and linear head

#### finetune backbone as well and linear head

### Generate learning curve for article

In [ ]:
with open('weights_train_uni_smallLRbackbone_complexhead_level4_nonorm_bracs_100epochs/history.json', 'r') as f:
    nonorm_curve_data = json.load(f)

In [ ]:
with open('weights_train_uni_smallLRbackbone_complexhead_level4_macenko_bracs_100epochs/history.json', 'r') as f:
    macenko_curve_data = json.load(f)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, axes = plt.subplots(1, 2, figsize=(8, 4), dpi=100)


# First subplot
axes[0].plot(np.arange(len(macenko_curve_data['train_loss'])), macenko_curve_data['train_loss'], label='train')
axes[0].plot(np.arange(len(macenko_curve_data['val_loss'])), macenko_curve_data['val_loss'], label='val')
axes[0].legend(fontsize=10)
axes[0].set_xlabel('Epochs', fontsize=12)
axes[0].set_ylabel('CE loss', fontsize=12)
axes[0].tick_params(labelsize=12)
# Add more xticks for the first subplot
axes[0].set_xticks(np.arange(0, len(macenko_curve_data['train_loss'])+1, 25))
axes[0].set_yticks([0.8, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0])
axes[0].text(0.5, -0.26, 'a)', transform=axes[0].transAxes, fontsize=12)

# Second subplot

# Second subplot
axes[1].plot(np.arange(len(nonorm_curve_data['train_loss'])), nonorm_curve_data['train_loss'], label='train')
axes[1].plot(np.arange(len(nonorm_curve_data['val_loss'])), nonorm_curve_data['val_loss'], label='val')
axes[1].legend(fontsize=10)
axes[1].set_xlabel('Epochs', fontsize=12)
axes[1].set_ylabel('CE loss', fontsize=12)
axes[1].tick_params(labelsize=12)
# Add more xticks for the second subplot
axes[1].set_xticks(np.arange(0, len(nonorm_curve_data['train_loss'])+1, 25))
axes[1].set_yticks([0.8, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0])
axes[1].text(0.5, -0.26, 'b)', transform=axes[1].transAxes, fontsize=12)

plt.tight_layout()  # Adjust the spacing between subplots if needed
#plt.show()
plt.savefig('../../article_plots/bracs_training_curves.svg', format='svg')
plt.savefig('../../article_plots/bracs_training_curves.jpg', format='jpg')

### Generating confusion matrix plots


In [ ]:
from torch.utils.data import DataLoader, TensorDataset, Dataset
import seaborn as sns

In [ ]:
def default_transforms(mean = (0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)):
    t = transforms.Compose(
                        [transforms.ToTensor(),
                         transforms.Normalize(mean = mean, std = std)])
    return t

##### Nonorm

In [ ]:
!ls | grep nonorm

In [ ]:
#weight_to_use = 'weights_train_uni_frozenbackbone_complexhead_level4_nonorm_bracs_300epochs/checkpoint_epoch_208_1.0834CE_61_acc.pth' # nonorm
weight_to_use = 'weights_train_uni_smallLRbackbone_complexhead_level4_nonorm_bracs_100epochs/checkpoint_epoch_70_1.0144CE_62_acc.pth'#checkpoint_epoch_65_1.0277CE_63_acc.pth'
loaded_model = torch.load( weight_to_use, map_location=torch.device('cuda'))
model.load_state_dict(loaded_model['model'])
model.eval();

In [ ]:
X_val.shape, y_val.shape

In [ ]:
# Create a torch tensor for the images
imgs_tensor_nonorm = torch.zeros((X_val.shape[0], X_val.shape[3], X_val.shape[2], X_val.shape[1]), dtype=torch.float32)

# Apply transforms to images
for i in range(X_val.shape[0]):
    imgs_tensor_nonorm[i] = default_transforms()(X_val[i])

In [ ]:
imgs_tensor_nonorm.shape

In [ ]:
bracs_class_pred_val_all_nonorm = []

# Create a data loader for the images
data_loader = DataLoader( imgs_tensor_nonorm, batch_size=128, shuffle=False, num_workers=1)

# Process the images using the feature extractor
for img in data_loader:
    img = img.to(device)

    with torch.no_grad():
        bracs_class_pred_val_nonorm = torch.nn.functional.softmax( model(img), dim=1 ) # softmax applied here !
        #bracs_class_pred_val = torch.max( model(img), dim=1 ) # softmax applied here !

    bracs_class_pred_val_all_nonorm.append(bracs_class_pred_val_nonorm.cpu().numpy())
bracs_class_pred_val_all_nonorm =  np.concatenate( bracs_class_pred_val_all_nonorm )

In [ ]:
print(classification_report( y_val, np.argmax( bracs_class_pred_val_all_nonorm, axis=1) ))

In [ ]:
# calculating the confusion matrix
confusion_matrix = pd.crosstab( y_val, np.argmax(bracs_class_pred_val_all_nonorm, axis=1),
                                rownames=['Real type'], colnames=['Predicted type'] )
#print( confusion_matrix )

# visualizng it on a heatmap
plt.figure(figsize=(12,12))
plt.title( 'Confusion matrix of ResNet50 - nonorm')
sns.heatmap(confusion_matrix, annot=True, xticklabels=np.array(lb.classes_), 
            yticklabels=np.array(lb.classes_), annot_kws={"size": 20} )
plt.tick_params(labelsize=12, rotation=25)
plt.xlabel('Predicted type')
plt.show()

##### Macenko

In [ ]:
#weight_to_use = 'weights_train_uni_level4_macenkonorm_bracs_300epochs/checkpoint_epoch_83_1.1439CE_60_acc.pth' # macenko based on bracs
## was pretty good: 
weight_to_use = 'weights_train_uni_smallLRbackbone_complexhead_level4_macenko_bracs_100epochs/checkpoint_epoch_70_1.0627CE_61_acc.pth' # macenko based on bracs
#weight_to_use = 'weights_train_uni_smallLRbackbone_level4_macenkonorm_bracs_300epochs_OLD/checkpoint_epoch_158_1.0442CE_62_acc.pth' # macenko based on bracs

loaded_model = torch.load( weight_to_use, map_location=torch.device('cuda'))
model.load_state_dict(loaded_model['model'])
model.eval();

In [ ]:
# Create a torch tensor for the images
imgs_tensor_macenko = torch.zeros((X_val.shape[0], X_val.shape[3], X_val.shape[2], X_val.shape[1]), dtype=torch.float32)

# Apply transforms to images
for i in range(X_val.shape[0]):
    imgs_tensor_macenko[i] = default_transforms()(X_val_macenko[i])

In [ ]:
bracs_class_pred_val_all_macenko = []

# Create a data loader for the images
data_loader = DataLoader( imgs_tensor_macenko, batch_size=128, shuffle=False, num_workers=1)

# Process the images using the feature extractor
for img in data_loader:
    img = img.to(device)

    with torch.no_grad():
        bracs_class_pred_val_macenko = torch.nn.functional.softmax( model(img), dim=1 ) # softmax applied here !

    bracs_class_pred_val_all_macenko.append(bracs_class_pred_val_macenko.cpu().numpy())
bracs_class_pred_val_all_macenko =  np.concatenate( bracs_class_pred_val_all_macenko )

In [ ]:
# calculating the confusion matrix
confusion_matrix_macenko = pd.crosstab( y_val, np.argmax(bracs_class_pred_val_all_macenko, axis=1),
                                rownames=['Real type'], colnames=['Predicted type'] )
#print( confusion_matrix )

# visualizng it on a heatmap
plt.figure(figsize=(12,12))
plt.title( 'Confusion matrix of ResNet50 - macenko')
sns.heatmap(confusion_matrix_macenko, annot=True, xticklabels=np.array(lb.classes_), 
            yticklabels=np.array(lb.classes_), annot_kws={"size": 20},  )
plt.tick_params(labelsize=12, rotation=25)
plt.xlabel('Predicted type')
plt.show()

In [ ]:
mycmap = plt.cm.get_cmap('bwr').reversed()
# visualizng it on a heatmap
plt.figure(figsize=(12,12))
plt.title( 'Confusion matrix of ResNet50')
sns.heatmap(confusion_matrix-confusion_matrix_macenko, annot=True, xticklabels=np.array(lb.classes_), 
            yticklabels=np.array(lb.classes_), annot_kws={"size": 20}, cmap=mycmap, vmin=-15, vmax=15 )
plt.tick_params(labelsize=12, rotation=25)
plt.xlabel('Predicted type')
plt.show()

In [ ]:
# normalize the confusion matrix
confusion_matrix_nonorm_percent = confusion_matrix.div(confusion_matrix.sum(axis=1), axis=0)

# visualizing the normalized confusion matrix on a heatmap
plt.figure(figsize=(12, 12))
plt.title('Normalized Confusion matrix of ResNet50 - nonorm')
sns.heatmap(confusion_matrix_nonorm_percent, annot=True, xticklabels=np.array(lb.classes_), 
            yticklabels=np.array(lb.classes_), annot_kws={"size": 20})
plt.tick_params(labelsize=12, rotation=25)
plt.xlabel('Predicted type')
plt.show()


In [ ]:
# normalize the confusion matrix
confusion_matrix_macenko_percent = confusion_matrix_macenko.div(confusion_matrix_macenko.sum(axis=1), axis=0)

# visualizing the normalized confusion matrix on a heatmap
plt.figure(figsize=(12, 12))
plt.title('Normalized Confusion matrix of ResNet50 - macenko')
sns.heatmap(confusion_matrix_macenko_percent, annot=True, xticklabels=np.array(lb.classes_), 
            yticklabels=np.array(lb.classes_), annot_kws={"size": 20})
plt.tick_params(labelsize=12, rotation=25)
plt.xlabel('Predicted type')
plt.show()


In [ ]:
# normalize the confusion matrix
confusion_matrix_macenko_percent = confusion_matrix_macenko.div(confusion_matrix_macenko.sum(axis=1), axis=0)

# calculate percentage deviations from the non-normalized confusion matrix
percentage_deviations = ( ( confusion_matrix_macenko_percent / confusion_matrix_nonorm_percent ) - 1 ) *100

# visualizing the normalized confusion matrix on a heatmap with percentage deviations
plt.figure(figsize=(12, 12))
plt.title('Normalized Confusion matrix of ResNet50 - macenko with Percentage Deviations')
sns.heatmap(confusion_matrix_macenko_percent, annot=True, xticklabels=np.array(lb.classes_), 
            yticklabels=np.array(lb.classes_), annot_kws={"size": 20}, cmap='viridis')
plt.tick_params(labelsize=12, rotation=25)

# add percentage deviations to the top right of each tile
for i in range(confusion_matrix_macenko_percent.shape[0]):
    for j in range(confusion_matrix_macenko_percent.shape[1]):

        if i == j:
            text = f'{np.round(percentage_deviations.iloc[i, j],2)}%'
            plt.text(j + 0.7, i + 0.1, text, fontsize=12, ha='center', va='center', color='green' if percentage_deviations.iloc[i, j] >= 0 else 'red' )
        
        else:
            text = f'{-np.round(percentage_deviations.iloc[i, j],2)}%'
            plt.text(j + 0.7, i + 0.1, text, fontsize=12, ha='center', va='center', color='green' if percentage_deviations.iloc[i, j] < 0 else 'red' )

            
plt.xlabel('Predicted type')
plt.show()


In [ ]:
print(classification_report( y_val, np.argmax( bracs_class_pred_val_all_nonorm, axis=1) ))

In [ ]:
print(classification_report( y_val, np.argmax( bracs_class_pred_val_all_macenko, axis=1) ))

In [ ]:
acc = 0
for n in range(confusion_matrix_nonorm_percent.shape[0]):
    acc += confusion_matrix_nonorm_percent.iloc[n,n]
acc /= n+1
acc

In [ ]:
acc = 0
for n in range(confusion_matrix_macenko_percent.shape[0]):
    acc += confusion_matrix_macenko_percent.iloc[n,n]
acc /= n+1
acc

In [ ]:
lb.classes_

In [ ]:
# Calculate and print individual class accuracies
class_accuracies = []
for i in range(confusion_matrix_nonorm_percent.shape[0]):
    # Since the matrix already contains percentages of correct predictions in the diagonal,
    # we directly take those as class accuracies.
    class_accuracy = confusion_matrix_nonorm_percent.iloc[i, i]
    class_accuracies.append(class_accuracy)
    print(f"Accuracy for {lb.classes_[i]}: {np.round(class_accuracy*100,2)}%")

# Overall accuracy
overall_acc = sum(class_accuracies) / len(class_accuracies)
print(f"\nOverall Accuracy: {overall_acc}%")

In [ ]:
# Calculate and print individual class accuracies
class_accuracies = []
for i in range(confusion_matrix_macenko_percent.shape[0]):
    # Since the matrix already contains percentages of correct predictions in the diagonal,
    # we directly take those as class accuracies.
    class_accuracy = confusion_matrix_macenko_percent.iloc[i, i]
    class_accuracies.append(class_accuracy)
    print(f"Accuracy for {lb.classes_[i]}: {np.round(class_accuracy*100,2)}%")

# Overall accuracy
overall_acc = sum(class_accuracies) / len(class_accuracies)
print(f"\nOverall Accuracy: {overall_acc}%")